# System deployment
In this laboratory, the objective is to convert a DDoS dection script written for Jupyter notebook into a stand-alone Python program that can be deployed to the target machine. To use this notebeook, first train and save an ANN model with the script of laboratory [03-Hyperparameters](https://github.com/doriguzzi/dl-for-network-security-phd/tree/main/03-Hyperparameters), and then test your model using this script.
If everything works as expected, convert this notebook into a Python script that supports the following command-line arguments:
- Path to the ANN model
- Path to the test set
- Path to the output CSV file where the program writes the classification results.

In [ ]:
# Author: Roberto Doriguzzi-Corin
# Project: Lecture on Intrusion Detection with Deep Learning
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import random as rn
import h5py
import glob
import time
import sys
import csv
import os
import logging
import pprint
from sklearn.metrics import f1_score, accuracy_score,confusion_matrix
from util_functions import *
from traffic_processing import *
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
tf.get_logger().setLevel(logging.ERROR)

SEED=1

# Seed Random Numbers
os.environ['PYTHONHASHSEED']=str(SEED)
np.random.seed(SEED)
rn.seed(SEED)
tf.random.set_seed(SEED)
config = tf.compat.v1.ConfigProto(inter_op_parallelism_threads=1)

FIELDNAMES = ['Model','Time','Packets', 'Samples', 'DDOS%','Accuracy', 'F1Score','TPR','FPR','TNR','FNR','Source']

In [ ]:
def report_results(Y_true, Y_pred,packets, model_name, data_source, prediction_time,writer):
    ddos_rate = '{:04.3f}'.format(sum(Y_pred)/Y_pred.shape[0])

    if Y_true is not None: # if we have the labels, we can compute the classification accuracy
        Y_true = Y_true.reshape((Y_true.shape[0], 1))
        accuracy = accuracy_score(Y_true, Y_pred)

        f1 = f1_score(Y_true, Y_pred)
        tn, fp, fn, tp = confusion_matrix(Y_true, Y_pred,labels=[0,1]).ravel()
        tnr = tn / (tn + fp)
        fpr = fp / (fp + tn)
        fnr = fn / (fn + tp)
        tpr = tp / (tp + fn)

        row = {'Model': model_name, 'Time': '{:04.3f}'.format(prediction_time), 'Packets': packets,
               'Samples': Y_pred.shape[0], 'DDOS%':ddos_rate,'Accuracy':accuracy, 'F1Score':f1,
               'TPR':tpr, 'FPR':fpr, 'TNR':tnr, 'FNR':fnr, 'Source':data_source}
    else:
        row = {'Model': model_name, 'Time': '{:04.3f}'.format(prediction_time), 'Packets': packets,
               'Samples': Y_pred.shape[0], 'DDOS%': ddos_rate, 'Accuracy': "N/A", 'F1Score': "N/A",
               'TPR': "N/A", 'FPR': "N/A", 'TNR': "N/A", 'FNR': "N/A", 'Source': data_source}
    pprint.pprint(row,sort_dicts=False)
    writer.writerow(row)

## External files
In the cell below, you need to set the right path to your ANN model. The path to the dataset is already set. However, the stand-alone Python program must take the two paths as command-line arguments, therefore the code in the following cell must be modified accordingly (e.g., using [argparse](https://docs.python.org/3/library/argparse.html)). 

In [ ]:
model = load_model("/path/to/the/mlp-model")
print (model.summary())
testset = "../Datasets/IDS2017/10t-10n-IDS2017-dataset-test.hdf5" # test set
X, Y = load_dataset(testset)

## Log file
In the stand-alone Python script, the path to the output log file must be specified as a command-line argument.

In [ ]:
predict_file = open('./results.csv', 'a', newline='')
predict_file.truncate(0)  # clean the file content (as we open the file in append mode)
predict_writer = csv.DictWriter(predict_file, fieldnames=FIELDNAMES)
predict_writer.writeheader()
predict_file.flush()

In [ ]:
iterations = 10

filename = testset.split('/')[-1].strip()
[packets] = count_packets_in_dataset([X])

Y_pred = None
Y_true = Y
avg_time = 0
for iteration in range(iterations):
    pt0 = time.time()
    Y_pred = np.squeeze(model.predict(X, batch_size=2048) > 0.5)
    pt1 = time.time()
    avg_time += pt1 - pt0

avg_time = avg_time / iterations

try:
    report_results(np.squeeze(Y_true), Y_pred, packets, model.name, filename, avg_time,predict_writer)
    predict_file.flush()
except:
    print("No packets received during the last time window.")

predict_file.close()